%pip uninstall opencv-python-headless -y 

%pip install opencv-python --upgrade

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,Cars64.xml,120,325,187,220
1,Cars233.xml,189,259,55,78
2,Cars243.xml,129,465,174,235
3,Cars10.xml,140,303,5,148
4,Cars203.xml,116,271,112,165


In [3]:
df.shape

(433, 5)

In [4]:
import xml.etree.ElementTree as xet

In [5]:
filename = df['filepath'][0]
filename

'Cars64.xml'

In [6]:
def getFilename(filename):
    filename_image = xet.parse(os.path.join('./images',filename)).getroot().find('filename').text
    # filepath_image = os.path.join('./images',filename_image)
    return os.path.join('./images', filename_image)

In [7]:
getFilename(filename)

'./images\\Cars64.png'

In [8]:
image_path = list(df['filepath'].apply(getFilename))
image_path 

['./images\\Cars64.png',
 './images\\Cars233.png',
 './images\\Cars243.png',
 './images\\Cars10.png',
 './images\\Cars203.png',
 './images\\Cars27.png',
 './images\\Cars8.png',
 './images\\Cars177.png',
 './images\\Cars249.png',
 './images\\Cars247.png',
 './images\\Cars145.png',
 './images\\Cars114.png',
 './images\\Cars69.png',
 './images\\Cars228.png',
 './images\\Cars202.png',
 './images\\Cars414.png',
 './images\\Cars221.png',
 './images\\Cars155.png',
 './images\\Cars196.png',
 './images\\Cars130.png',
 './images\\Cars296.png',
 './images\\Cars314.png',
 './images\\Cars86.png',
 './images\\Cars343.png',
 './images\\Cars115.png',
 './images\\Cars208.png',
 './images\\Cars142.png',
 './images\\Cars104.png',
 './images\\Cars15.png',
 './images\\Cars137.png',
 './images\\Cars125.png',
 './images\\Cars288.png',
 './images\\Cars132.png',
 './images\\Cars332.png',
 './images\\Cars194.png',
 './images\\Cars318.png',
 './images\\Cars158.png',
 './images\\Cars400.png',
 './images\\Cars116.

#### verify image and output

In [9]:
file_path = image_path[0]
file_path 

'./images\\Cars64.png'

In [10]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# 1093	1396	645	727
cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Preprocessing

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [13]:
labels = df.iloc[:,1:].values

In [14]:
type(image_path[0])

str

In [15]:
data = []
output = []
# for i in os.listdir()
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [16]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [17]:
X.shape,y.shape

((433, 224, 224, 3), (433, 4))

In [18]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

### Deep Learning Model

In [19]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [ ]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

 21889024/219055592 [=>............................] - ETA: 6:52

In [ ]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

### model training

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
tfb = TensorBoard('object_detection')

In [ ]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

In [ ]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

In [ ]:
model.save('./models/object_detection.h5')